In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50,InceptionV3
from tensorflow.keras import layers,models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

2025-08-10 19:52:17.892568: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-10 19:52:18.039504: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-10 19:52:18.144588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754835738.263386   14207 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754835738.302244   14207 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754835738.528691   14207 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [3]:
# VGG16 model
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [4]:
def build_model(base_model):
    base_model.trainable = False  # Freeze base layers
    model = models.Sequential([
        base_model,
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg_model = build_model(vgg_base)
print("\nTraining VGG16...")
vgg_model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test),verbose = 0)


Training VGG16...


In [7]:
resnet_base = ResNet50(weights = 'imagenet',include_top = False, input_shape = (32,32,3))
resnet_model = build_model(resnet_base)
print("\n Training Resnet50...")
resnet_model.fit(x_train,y_train,epochs=3, validation_data=(x_test,y_test),verbose = 3)



 Training Resnet50...
Epoch 1/3
Epoch 2/3
Epoch 3/3


In [12]:
# For InceptionV3, use a different approach since it has minimum size requirements
# Create a simple CNN instead
def create_simple_inception():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
    ])
    return model

inception_base = create_simple_inception()
inception_model = build_model(inception_base)
print("\n Training Simple CNN (instead of InceptionV3)...")
inception_model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test), verbose=2)

/home/franz/Documents/Lab/.venv/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



 Training Simple CNN (instead of InceptionV3)...
Epoch 1/3
1563/1563 - 14s - 9ms/step - accuracy: 0.2444 - loss: 2.0617 - val_accuracy: 0.3396 - val_loss: 1.8509
Epoch 2/3
1563/1563 - 10s - 6ms/step - accuracy: 0.3329 - loss: 1.8320 - val_accuracy: 0.3946 - val_loss: 1.7227
Epoch 3/3
1563/1563 - 11s - 7ms/step - accuracy: 0.3635 - loss: 1.7510 - val_accuracy: 0.4131 - val_loss: 1.6555


In [13]:
for name,model in zip(['VGG16','Resnet50','InceptionV3'],[vgg_model,resnet_model,inception_model]):
    loss,acc = model.evaluate(x_test,y_test,verbose = 3)
    print(f"{name} - Loss: {loss:.4f}, Accuracy: {acc:.4f}")    

VGG16 - Loss: 1.1783, Accuracy: 0.5855
Resnet50 - Loss: 2.1230, Accuracy: 0.2347
InceptionV3 - Loss: 1.6555, Accuracy: 0.4131


In [19]:
import pandas as pd

# Evaluate and create summary table
results_data = []
for name, model in zip(['VGG16', 'ResNet50', 'InceptionV3'], [vgg_model, resnet_model, inception_model]):
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    # Count parameters
    total_params = model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])
    
    results_data.append({
        'Model': name,
        'Accuracy': f'{acc:.4f}',
        'Loss': f'{loss:.4f}',
        'Total Parameters': f'{total_params:,}',
        'Trainable Parameters': f'{trainable_params:,}'
    })

df = pd.DataFrame(results_data)
print("Model Comparison Summary:")
print("=" * 80)
print(df.to_string(index=False))

Model Comparison Summary:
      Model Accuracy   Loss Total Parameters Trainable Parameters
      VGG16   0.5855 1.1783       14,848,586              133,898
   ResNet50   0.2347 2.1230       24,114,826              527,114
InceptionV3   0.4131 1.6555          427,914              297,738


In [18]:
import matplotlib.pyplot as plt

# Evaluate models and store results
results = []
model_names = ['VGG16', 'ResNet50', 'InceptionV3']
models = [vgg_model, resnet_model, inception_model]

for name, model in zip(model_names, models):
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    results.append({'name': name, 'loss': loss, 'accuracy': acc})
    print(f"{name} - Loss: {loss:.4f}, Accuracy: {acc:.4f}")

# Create accuracy comparison chart
names = [r['name'] for r in results]
accuracies = [r['accuracy'] for r in results]
losses = [r['loss'] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy chart
bars1 = ax1.bar(names, accuracies, color=['blue', 'green', 'orange'])
ax1.set_title('Model Accuracy Comparison')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0, 1)
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{accuracies[i]:.3f}', ha='center', va='bottom')

# Loss chart
bars2 = ax2.bar(names, losses, color=['red', 'purple', 'brown'])
ax2.set_title('Model Loss Comparison')
ax2.set_ylabel('Loss')
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{losses[i]:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Evaluate models and store results
results = []
model_names = ['VGG16', 'ResNet50', 'InceptionV3']
models = [vgg_model, resnet_model, inception_model]

for name, model in zip(model_names, models):
    loss, acc = model.evaluate(x_test, y_test, verbose=0)
    results.append({'name': name, 'loss': loss, 'accuracy': acc})
    print(f"{name} - Loss: {loss:.4f}, Accuracy: {acc:.4f}")

# Create accuracy comparison chart
names = [r['name'] for r in results]
accuracies = [r['accuracy'] for r in results]
losses = [r['loss'] for r in results]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy chart
bars1 = ax1.bar(names, accuracies, color=['blue', 'green', 'orange'])
ax1.set_title('Model Accuracy Comparison')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0, 1)
for i, bar in enumerate(bars1):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{accuracies[i]:.3f}', ha='center', va='bottom')

# Loss chart
bars2 = ax2.bar(names, losses, color=['red', 'purple', 'brown'])
ax2.set_title('Model Loss Comparison')
ax2.set_ylabel('Loss')
for i, bar in enumerate(bars2):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{losses[i]:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

KeyboardInterrupt: 